<a href="https://colab.research.google.com/github/arunmurali11/Intro/blob/master/FoodImageClassificationNew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

/bin/bash: conda: command not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.9 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-stvbtubi
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-stvbtubi
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369501 sha256=bc1104d5ba058fffe3b54aa6072f32a4b6a5e1a07878a14be47dd324e96f9443
  Stored in directory: /tmp/pip-ephem-wheel-cache-op53vjaf/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import auth
auth.authenticate_user()


In [6]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from PIL import Image
from io import BytesIO
import torch
import clip

# Load the CLIP model and pre-process text_list
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14", device=device)
text_list = ["pork", "friedRice", "bagelSandwich", "bread", "bibimBop", "applePie"]
text_inputs = clip.tokenize(text_list).to(device)

def download_file(file_id):
    service = build('drive', 'v3')
    request = service.files().get_media(fileId=file_id)
    file_io = BytesIO()
    downloader = MediaIoBaseDownload(file_io, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
    file_io.seek(0)
    return file_io

def list_files(folder_id):
    service = build('drive', 'v3')
    results = service.files().list(
        q=f"'{folder_id}' in parents",
        fields="files(id, name, mimeType)"
    ).execute()

    files = results.get('files', [])
    return files

def process_image(file_id, file_name):
    file_io = download_file(file_id)
    image = Image.open(file_io).convert('RGB')

    # Pre-process the image
    image_input = preprocess(image).unsqueeze(0).to(device)

    # Perform classification using CLIP
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_inputs)
        similarity_scores = (100.0 * image_features @ text_features.T).softmax(dim=-1)

    # Get the index of the top predicted class
    predicted_class_index = similarity_scores.argmax().item()
    predicted_class = text_list[predicted_class_index]

    return predicted_class

root_folder_id = '1fTBPKhOU5bTIo6gTmJmvzDXCT5fXUvTz'# Provide root folder id

service = build('drive', 'v3')
folders = list_files(root_folder_id)

subfolder_correct = {}
subfolder_incorrect = {}

incorrect_files = []  # To track incorrectly classified files

for folder in folders:
    folder_id = folder['id']
    folder_name = folder['name']
    print(f"Processing files in folder: {folder_name}")

    files = list_files(folder_id)

    correct_classifications = 0
    incorrect_classifications = 0

    for file in files:
        file_id = file['id']
        file_name = file['name']
        mime_type = file['mimeType']

        if mime_type.startswith('image/'):
            print(f"Processing image file: {file_name}")
            predicted_class = process_image(file_id, file_name)

            # Check if the predicted class matches the folder name
            if predicted_class.lower() == folder_name.lower():
                correct_classifications += 1
                print("Correctly classified")
            else:
                incorrect_classifications += 1
                print("Incorrectly classified")
                incorrect_files.append((folder_name, file_name))  # Add to the list of incorrectly classified files

    # Store the counts in the respective subfolder dictionaries
    subfolder_correct[folder_name] = correct_classifications
    subfolder_incorrect[folder_name] = incorrect_classifications

# Print the classification results for each subfolder
for folder_name, correct_count in subfolder_correct.items():
    incorrect_count = subfolder_incorrect[folder_name]
    print(f"\nSubfolder: {folder_name}")
    print(f"Correctly classified: {correct_count}")
    print(f"Incorrectly classified: {incorrect_count}")

# Print the incorrectly classified files
print("\nIncorrectly Classified Files:")
for folder_name, file_name in incorrect_files:
    print(f"Folder: {folder_name}, File: {file_name}")


100%|███████████████████████████████████████| 890M/890M [00:11<00:00, 78.3MiB/s]


Processing files in folder: Pork
Processing image file: B070220XX_11096.jpg
Correctly classified
Processing image file: B070220XX_11098.jpg
Correctly classified
Processing image file: B070220XX_11095.jpg
Correctly classified
Processing image file: B070220XX_11097.jpg
Correctly classified
Processing image file: B070220XX_11080.jpg
Correctly classified
Processing image file: B070220XX_11076.jpg
Correctly classified
Processing image file: B070220XX_11087.jpg
Correctly classified
Processing image file: B070220XX_11083.jpg
Correctly classified
Processing image file: B070220XX_11085.jpg
Correctly classified
Processing image file: B070220XX_11079.jpg
Correctly classified
Processing image file: B070220XX_11068.jpg
Correctly classified
Processing image file: B070220XX_11072.jpg
Correctly classified
Processing image file: B070220XX_11075.jpg
Correctly classified
Processing image file: B070220XX_11077.jpg
Correctly classified
Processing image file: B070220XX_11086.jpg
Correctly classified
Process

In [ ]:
# Subfolder: Pork
# Correctly classified: 88
# Incorrectly classified: 1

# Subfolder: Bibimbop
# Correctly classified: 86
# Incorrectly classified: 6

# Subfolder: BagelSandwich
# Correctly classified: 45
# Incorrectly classified: 0

# Subfolder: FriedRice
# Correctly classified: 85
# Incorrectly classified: 3

# Subfolder: ApplePie
# Correctly classified: 88
# Incorrectly classified: 5

# Subfolder: Bread
# Correctly classified: 89
# Incorrectly classified: 1

# Incorrectly Classified Files:
# Folder: Pork, File: B070220XX_11055.jpg
# Folder: Bibimbop, File: B010316XX_02440.jpg
# Folder: Bibimbop, File: B010316XX_02435.jpg
# Folder: Bibimbop, File: B010316XX_02434.jpg
# Folder: Bibimbop, File: B010316XX_02429.jpg
# Folder: Bibimbop, File: B010316XX_02428.jpg
# Folder: Bibimbop, File: B010316XX_02433.jpg
# Folder: FriedRice, File: B010443XX_10244.jpg
# Folder: FriedRice, File: B010443XX_10226.jpg
# Folder: FriedRice, File: B010443XX_10200.jpg
# Folder: ApplePie, File: A020511XX_02718.jpg
# Folder: ApplePie, File: A020511XX_02717.jpg
# Folder: ApplePie, File: A020511XX_02715.jpg
# Folder: ApplePie, File: A020511XX_02631.jpg
# Folder: ApplePie, File: A020511XX_02611.jpg
# Folder: Bread, File: A020118XX_11009.jpg

In [9]:
 (88+86+45+85+88+89)/497 #Accuracy = 96.78%

0.9678068410462777